# Google Analytics Capstone Project #

In [3]:
import pandas as pd
import glob
import plotly.express as px
import numpy as np
from pygeocoder import Geocoder

First will explore only one file (month) due to size of each file and try to find if any clues given in course description still applies to current data source.

In [4]:
df_sep = pd.read_csv("./source_data/202009-divvy-tripdata.csv")
df_sep.head()

,ride_id,rideable_type,started_at,ended_at,start_station_name,start_station_id,end_station_name,end_station_id,start_lat,start_lng,end_lat,end_lng,member_casual
0,2B22BD5F95FB2629,electric_bike,2020-09-17 14:27:11,2020-09-17 14:44:24,Michigan Ave & Lake St,52.0,Green St & Randolph St,112.0,41.886692,-87.623561,41.883570,-87.648731,casual
1,A7FB70B4AFC6CAF2,electric_bike,2020-09-17 15:07:31,2020-09-17 15:07:45,W Oakdale Ave & N Broadway,NaN,W Oakdale Ave & N Broadway,NaN,41.940000,-87.640000,41.940000,-87.640000,casual
2,86057FA01BAC778E,electric_bike,2020-09-17 15:09:04,2020-09-17 15:09:35,W Oakdale Ave & N Broadway,NaN,W Oakdale Ave & N Broadway,NaN,41.940000,-87.640000,41.940000,-87.640000,casual
3,57F6DC9A153DB98C,electric_bike,2020-09-17 18:10:46,2020-09-17 18:35:49,Ashland Ave & Belle Plaine Ave,246.0,Montrose Harbor,249.0,41.956060,-87.668916,41.963985,-87.638216,casual
4,B9C4712F78C1AE68,electric_bike,2020-09-17 15:16:13,2020-09-17 15:52:55,Fairbanks Ct & Grand Ave,24.0,Fairbanks Ct & Grand Ave,24.0,41.891860,-87.621008,41.891346,-87.620325,casual


In [7]:
df_sep.start_station_name.unique()

array(['Michigan Ave & Lake St', 'W Oakdale Ave & N Broadway',
       'Ashland Ave & Belle Plaine Ave', 'Fairbanks Ct & Grand Ave',
       'Clark St & Armitage Ave', 'Wells St & Evergreen Ave', nan,
       'Michigan Ave & 18th St', 'Mies van der Rohe Way & Chestnut St',
       'Halsted St & Polk St', 'Albany Ave & Bloomingdale Ave',
       'Lake Shore Dr & Diversey Pkwy', 'Elston Ave & Wabansia Ave',
       'Clybourn Ave & Division St', 'Campbell Ave & Fullerton Ave',
       'Clark St & Lake St', 'Sawyer Ave & Irving Park Rd',
       'Financial Pl & Ida B Wells Dr', 'Wells St & Huron St',
       'Shedd Aquarium', 'Streeter Dr & Grand Ave',
       'Bissell St & Armitage Ave', 'Franklin St & Lake St',
       'Sedgwick St & Huron St', 'Dearborn Pkwy & Delaware Pl',
       'Halsted St & Dickens Ave', 'Clark St & Schiller St',
       'Michigan Ave & Washington St', 'Aberdeen St & Jackson Blvd',
       'Wilton Ave & Diversey Pkwy', 'Wells St & Hubbard St',
       'Archer (Damen) Ave & 37th S

there are some entries with starting point HUBBARD ST BIKE CHECKING (LBS-WH-TEST), it could mean these were taken by company for repair. Course description suggest name HQ QR but there is nothing like that. It could have changed.

There is also WATTSON TESTING DIVVY - this could indicate another station that is used by the company for checking bikes.

In [17]:
df_sep = df_sep[(df_sep.start_station_name != "HUBBARD ST BIKE CHECKING (LBS-WH-TEST)")]
df_sep = df_sep[(df_sep.end_station_name != "HUBBARD ST BIKE CHECKING (LBS-WH-TEST)")]
df_sep = df_sep[(df_sep.start_station_name != "WATSON TESTING - DIVVY")]
df_sep = df_sep[(df_sep.end_station_name != "WATSON TESTING - DIVVY")]

In [20]:
df_sep = df_sep.drop(["start_station_name","start_station_id", "end_station_name", "end_station_id", "start_lat", "start_lng", "end_lat", "end_lng"], axis=1)

In [21]:
df_sep["started_at"] = pd.to_datetime(df_sep["started_at"])
df_sep["ended_at"] = pd.to_datetime(df_sep["ended_at"])

In [22]:
df_sep["ride_length"] = df_sep["ended_at"] - df_sep["started_at"]
df_sep["day_of_week"] = df_sep["started_at"].dt.day_name()

In [25]:
df_sep.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 532221 entries, 0 to 532957
Data columns (total 7 columns):
 #   Column         Non-Null Count   Dtype          
---  ------         --------------   -----          
 0   ride_id        532221 non-null  object         
 1   rideable_type  532221 non-null  object         
 2   started_at     532221 non-null  datetime64[ns] 
 3   ended_at       532221 non-null  datetime64[ns] 
 4   member_casual  532221 non-null  object         
 5   ride_length    532221 non-null  timedelta64[ns]
 6   day_of_week    532221 non-null  object         
dtypes: datetime64[ns](2), object(4), timedelta64[ns](1)
memory usage: 48.6+ MB


In [23]:
df_sep[df_sep["ride_length"]<= pd.Timedelta(seconds=0)]

,ride_id,rideable_type,started_at,ended_at,member_casual,ride_length,day_of_week
6168,8E30DCBB5E1C885B,electric_bike,2020-09-10 13:49:20,2020-09-10 13:49:20,casual,0 days 00:00:00,Thursday
13005,861CF45DB64109FB,electric_bike,2020-09-02 17:05:11,2020-09-02 17:05:11,casual,0 days 00:00:00,Wednesday
13473,1F7A089EE71EABAF,electric_bike,2020-09-05 16:11:11,2020-09-05 16:11:10,casual,-1 days +23:59:59,Saturday
21913,C01F07BA7B6E3650,electric_bike,2020-09-06 04:25:27,2020-09-06 04:25:27,casual,0 days 00:00:00,Sunday
22935,CCF83DF13C53140E,electric_bike,2020-09-06 22:05:44,2020-09-06 22:05:44,casual,0 days 00:00:00,Sunday
...,...,...,...,...,...,...,...
515338,FBBC2AE3DF2B8D85,docked_bike,2020-09-04 19:52:10,2020-09-04 19:51:58,member,-1 days +23:59:48,Friday
515343,DA30091B712C4BBF,docked_bike,2020-09-03 19:02:46,2020-09-03 19:02:35,member,-1 days +23:59:49,Thursday
517684,8714FA8188577CC2,electric_bike,2020-09-15 18:57:13,2020-09-15 18:57:13,casual,0 days 00:00:00,Tuesday
523219,4A09A29DAC49024A,electric_bike,2020-09-10 15:46:14,2020-09-10 15:46:14,casual,0 days 00:00:00,Thursday


From one file we learned there are two stations that shouldn't be included in our dataset. There are also some entries with 0 or negative duration. These will have to be removed from full dataset before analyzing differencec between users.

Let's read all csv files and creating one dataframe with all informations

In [ ]:
df_from_each_file = (pd.read_csv(f) for f in glob.glob("./source_data/*.csv"))
df = pd.concat(df_from_each_file, ignore_index=True)

Next we explore start and end of the dataframe


In [ ]:
df

We can see there are two columns with dates. Specificaly when bike ride started and when ended. We proceed with converting dates to correct datetime formats.

In [ ]:
df["started_at"] = pd.to_datetime(df["started_at"])
df["ended_at"] = pd.to_datetime(df["ended_at"])
df.info()

Next step will be to check if we have any missing data.

In [ ]:
df[df.isnull().any(axis=1)]

Most of the NaN are in station names and ids. We don't need these information because street address or uniqe station id can be determinated from lat and lng coordinates.

In [ ]:
df = df.drop(["start_station_name","end_station_name","start_station_id","end_station_id"], axis=1)
df[df.isnull().any(axis=1)]

There are still some misstin data in end station coordinates. Let's check if there are missing data in any other column by dropping end_lat and end_lng.

In [ ]:
df[df.drop(["end_lat", "end_lng"],axis=1).isnull().any(axis=1)]

Everything else looks good, we can now drop any duplicate rows and continute.

In [ ]:
df = df.drop_duplicates()
df.head()

With correct datetime formats we can calculate ride lenght and determine day of the week the ride took place.

In [ ]:
df["ride_length"] = df["ended_at"] - df["started_at"]
df["day_of_week"] = df["started_at"].dt.day_name()
df

For perfonmance issues we'll drop start_lat and start_lng

In [ ]:
df = df.drop(["start_lat", "start_lng","end_lat", "end_lng"],axis=1)

In [ ]:
print("Mean ride length: ", df.groupby("member_casual")["ride_length"].mean())
print("Max ride length: ",df.groupby("member_casual")["ride_length"].max())
print("Min ride length: ",df.groupby("member_casual")["ride_length"].min())
print("Mode day of the week: ",df.groupby("member_casual")["day_of_week"].agg(pd.Series.mode))

From first exploration of data we found shortest duration of ride is negative. This is no good as any rental can't have negative duration. Let's explore how many entries have negative or duration shorter then 30 seconds as it's very unlikely customer was able to get anywhere in less than 30 seconds.

In [ ]:
df[df["ride_length"]<pd.Timedelta(seconds=0)]

In [ ]:
df[df["ride_length"]<pd.Timedelta(seconds=30)]

We have noticed there is many rides with negative length and even more rides with length less than 30 sec. We have to ask why is it?

There could be couple reasons:
* Clocks on bikes/stations are out of sync and that results in wrong readings
* Positive ride length with very short duration might be due to customers changing their mind and returning it right away. Is there any charge for casual riders?

Quality of data shows some inconsictencies.

In [ ]:
fid = px.histogram(df[(df["ride_length"] >= pd.Timedelta(seconds=-30)) & (df["ride_length"] < pd.Timedelta(seconds=30))], x = "ride_length", nbins = 20)

In [ ]:
fid.show()

How much do short rides skew mean?

In [ ]:
df[df["ride_length"] >= pd.Timedelta(seconds=30)].groupby("member_casual")["ride_length"].mean()

In [ ]:
df.groupby("member_casual")["ride_length"].mean()

Impact of negative length is bigger for members. But we have to keep in mind, these don't have to be only wrong data. In fact all entries can have wrong start and end time to same degree which can't be now determined. All analysis could be completely wrong so this should be checked more in depth.

In [ ]:
px.histogram(df[(df["ride_length"] >= pd.Timedelta(seconds=-1800)) & (df["ride_length"] < pd.Timedelta(seconds=7200))& (df["member_casual"]=="member")], x="ride_length", nbins=300).show()

In [ ]:
df.head()